In [4]:
import os
import nltk
import io
import speech_recognition as sr

# you'll need to run nltk.download() and download all packages
#nltk.download_shell()
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.data.path.append('D:\\JacobSchool\\CS540\\nltk_data')
from nltk.corpus import stopwords
from nltk.probability import ConditionalFreqDist

In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize

Sets from http://www.iaees.org/publications/journals/selforganizology/articles/2016-3(3)/algorithm-to-transform-natural-language-into-SQL-queries.pdf
Escape Word Set<br/>
Expression Mapping Set<br/>
Noun Set<br/>
Verb Set<br/>
Semantic Set<br/>
Variable Set<br/>
Relation Set<br/>
Attribute Set<br/>
Conjunction Set<br/>
This is all the sets for words and rules:

In [6]:
escape_words = set(stopwords.words('english'))#["a", "an", "the", "which", "is", "of", "with", "to", "for", "are", "and", "should", "be"]

rules_on_top_of = []

rules_side_by_side = []

nouns = ["block", "wildcard", "block0", "block1", "block2", "block3", "block4", "block5", "block6", "block7", "block8", "block9" \
         "block10", "block11", "block12", "block13", "block14", "block15", "block16", "block17", "block18", "block19" \
         "block20", "block21", "block22", "block23", "block24", "block25", "block26", "block27", "block28", "block29" \
         "block30", "block31", "block32", "block33", "block34", "block35", "block36", "block37", "block38", "block39" \
         "block40", "block41", "block42", "block43", "block44", "block45", "block46", "block47", "block48", "block49" \
         "block50", "block51", "block52", "block53", "block54", "block55", "block56", "block57", "block58", "block59" \
         "block60", "block61", "block62", "block63", "block64", "block65", "block66", "block67", "block68", "block69" \
         "block70", "block71", "block72", "block73", "block74", "block75", "block76", "block77", "block78", "block79" \
         "block80", "block81", "block82", "block83", "block84", "block85", "block86", "block87", "block88", "block89" \
         "block90", "block91", "block92", "block93", "block94", "block95", "block96", "block97", "block98", "block99" \
         "wildcard0", "wildcard1", "wildcard2", "wildcard3", "wildcard4", "wildcard5", "wildcard6", "wildcard7", "wildcard8", "wildcard9" \
         "wildcard10", "wildcard11", "wildcard12", "wildcard13", "wildcard14", "wildcard15", "wildcard16", "wildcard17", "wildcard18", "wildcard19" \
         "wildcard20", "wildcard21", "wildcard22", "wildcard23", "wildcard24", "wildcard25", "wildcard26", "wildcard27", "wildcard28", "wildcard29" \
         "wildcard30", "wildcard31", "wildcard32", "wildcard33", "wildcard34", "wildcard35", "wildcard36", "wildcard37", "wildcard38", "wildcard39" \
         "wildcard40", "wildcard41", "wildcard42", "wildcard43", "wildcard44", "wildcard45", "wildcard46", "wildcard47", "wildcard48", "wildcard49" \
         "wildcard50", "wildcard51", "wildcard52", "wildcard53", "wildcard54", "wildcard55", "wildcard56", "wildcard57", "wildcard58", "wildcard59" \
         "wildcard60", "wildcard61", "wildcard62", "wildcard63", "wildcard64", "wildcard65", "wildcard66", "wildcard67", "wildcard68", "wildcard69" \
         "wildcard70", "wildcard71", "wildcard72", "wildcard73", "wildcard74", "wildcard75", "wildcard76", "wildcard77", "wildcard78", "wildcard79" \
         "wildcard80", "wildcard81", "wildcard82", "wildcard83", "wildcard84", "wildcard85", "wildcard86", "wildcard87", "wildcard88", "wildcard89" \
         "wildcard90", "wildcard91", "wildcard92", "wildcard93", "wildcard94", "wildcard95", "wildcard96", "wildcard97", "wildcard98", "wildcard99" ]

verbs_top = ["top", "above", "over", "on"]
verbs_below = ["below", "underneath",  "under"]
verbs_side = ["side", "neighboring", "beside", "next"]

variables_colors = ["amber", "amethyst", "apricot", "aquamarine", "azure", "beige", "black", "blue", "blush", "bronze", "brown", "burgundy", "byzantium", "carmine", "cerise", "cerulean", "champagne", "chocolate", "coffee", "copper", "coral", "crimson", "cyan", "emerald", "erin", "gold", "gray", "green", "harlequin", "indigo", "ivory", "jade", "lavender", "lemon", "lilac", "lime", "magenta", "maroon", "mauve", "navy", "ochre", "olive", "orange", "orchid", "peach", "pearl", "periwinkle", "pink", "plum", "puce", "purple", "raspberry", "red", "rose", "ruby", "salmon", "sangria", "sapphire", "scarlet", "silver", "tan", "taupe", "teal", "turquoise", "ultramarine", "violet", "viridian", "white", "yellow"]
#variables_location = ["0","zero","1", "one", "2", "two", "3", "three", "4", "four", "5", "five", "6", "six", "7", "seven", "8", "eight", "9", "nine"]
loc_options = ["move", "location", "spot", "position", "address", "place", "locality", "point", "placement", "locale", "setting", "bearings", "bearing", "venue", "is", "at"]

#position, place, situation, site, locality, locale, spot, whereabouts, point, placement; scene, setting, area, environment; bearings, orientation; venue, address;

Custom Tags:<br/>
NN &rightarrow; Noun<br/>
VT &rightarrow; Verb Top<br/>
VB &rightarrow; Verb Below<br/>
VS &rightarrow; Verb Side<br/>
VarC &rightarrow; Variable Color<br/>
VarL &rightarrow; Variable Location<br/>

In [7]:
def enumerate_parts(parts_of_speech):
    
    #for p in parts_of_speech:
    for i,w in enumerate(parts_of_speech):
        if(w[0] in nouns):
            tw = list(w)
            tw[1]='BNN'
            w=tuple(tw)
            parts_of_speech[i]=w
        elif(w[0] in variables_colors):
            tw = list(w)
            tw[1]='VARC'
            w=tuple(tw)
            parts_of_speech[i]=w
        elif(w[0] in verbs_top):
            tw = list(w)
            tw[1]='VT'
            w=tuple(tw)
            parts_of_speech[i]=w
        elif(w[0] in verbs_below):
            tw = list(w)
            tw[1]='VBOT' #Changed to a value that is not in the default parse list
            w=tuple(tw)
            parts_of_speech[i]=w
        elif(w[0] in verbs_side):
            tw = list(w)
            tw[1]='VS'
            w=tuple(tw)
            parts_of_speech[i]=w
        elif(w[0] in loc_options):
            tw = list(w)
            tw[1]='VARL'
            w=tuple(tw)
            parts_of_speech[i]=w
        #Not needed the CD tag represents a number
        #elif(w[0] in variables_numbers):
        #    tw = list(w)
        #    tw[1]='INT'
        #    w=tuple(tw)
        #    p[i]=w
    #return parts_of_speech

In [8]:
def processLocation(tagged_sentence):
        m_grammar = """LOCATION: {<VARL>.*<.*>.*<CD><CD><CD>}
                                {<CD><CD><CD>}"""
        chunkParser = nltk.RegexpParser(m_grammar)
        tree = chunkParser.parse(tagged_sentence)
        location = []
        for subtree in tree.subtrees():
            if subtree.label() == "LOCATION":
                loc_values = subtree.leaves()[-3:]
                location.append(loc_values[0][0])
                location.append(loc_values[1][0])
                location.append(loc_values[2][0])
        
        #print("LOCATION: "+ str(location))
        
        return location

In [9]:
def checkBlocks(parts_of_speech):
        m_grammar = """BLOCK:  {<BNN>.*<CD>}
                                {<BNN>?}"""
        chunkParser = nltk.RegexpParser(m_grammar)
        tree = chunkParser.parse(parts_of_speech)

        blocks=[]
        
        #Collect the two blocks out of the parts of speech
        for subtree in tree.subtrees():
            if subtree.label() == "BLOCK":
                #print("BLOCK: "+str(subtree.leaves()))                
                block_1 = subtree.leaves()[0][0]
                if(len(subtree.leaves())>1):
                    block_1 += subtree.leaves()[1][0]
                    #print("Block_1: " + block_1)
                blocks.append(block_1)
        #print("The Blocks: " + str(blocks))
        return blocks

In [10]:
def setRelation(parts_of_speech):
    m_grammar = """ONTOP: {<VT>.*}
                        BOTTOM: {<VBOT>.*}
                        SIDE: {<VS>.*}"""
    
    #Need to see if there is an and between the relations
    
    chunkParser = nltk.RegexpParser(m_grammar)
    tree = chunkParser.parse(parts_of_speech)
    
    #swap_on_top_of = False
    relations=[]
    for subtree in tree.subtrees():
        if subtree.label() == "ONTOP" or subtree.label() == "BOTTOM":
            if subtree.label() == "BOTTOM":
                #swap_on_top_of = True
                relations.append("bottom-of")
            else:
                relations.append("on-top-of")
            #print("ONTOP: "+str(subtree.leaves()))
        if subtree.label() == "SIDE":
            relations.append("side-by-side")
            #print("SIDE: "+str(subtree.leaves()))
    
    #print("The Relations: " + str(relations))
    return relations#,swap_on_top_of

In [11]:
def processColor(parts_of_speech):
    color=""
    m_grammar = """COLOR: {<VARC.*>}"""
    
    chunkParser = nltk.RegexpParser(m_grammar)
    tree = chunkParser.parse(parts_of_speech)
    for subtree in tree.subtrees():
        if subtree.label() == "COLOR":
                #print("COLOR: "+str(subtree.leaves()))
                color = subtree.leaves()[0][0]
                
    #print("The Color: " + color)
    return color

In [12]:
def chunkParts(p):
    m_grammar = """ONTOP: {<VT>.*}
                    BOTTOM: {<VBL>.*}
                    LOCATION: {<VARL>.*<.*>.*<CD>.*<CD>.*<CD>}
                            {<CD>.*<CD>.*<CD>}
                    SIDE: {<VS>.*}
                    BLOCK:  {<BNN>.*<CD>}
                            {<BNN>?}
                    COLOR: {<VARC.*>}"""
    #print(p)
    chunkParser = nltk.RegexpParser(m_grammar)
    return chunkParser.parse(p)

In [13]:
def processAndInSentence(parts_of_speech):
    m_grammar = """AND: {<CC>}"""
    
    chunkParser = nltk.RegexpParser(m_grammar)
    tree = chunkParser.parse(parts_of_speech)
    hasAnd = False
    
    for subtree in tree.subtrees():
        if subtree.label() == "AND":
                #print("AND: "+ str(subtree.leaves()))
                hasAnd = True

    return hasAnd

In [14]:
def format_command(outputfile, blocks, relations, color, location, hasAnd):
    #print("format_command")
    #print(blocks)
    #print(relations)
    #print(color)
    #print(location)
    #print(swap_on_top_of)
    try:
        thecommand = ""
        block1=""
        block2=""
        relation=""
        relation2=""

        #Set the first block
        if len(blocks) > 0:
            block1=blocks[0]
        
        #Set the first relation
        if len(relations) > 0:
            relation=relations[0]
            
        #Multiple blocks and one relations
        if (len(blocks)>1) and (len(blocks)<4) and len(relations)>0:
            for i, block in enumerate(blocks):
                if i > 0:
                    block2=block

                if block1 and block2 and relation:
                    if relations[0] == "bottom-of":#
                        thecommand += "(is %s %s %s)\n" %(block2, block1, "on-top-of")
                    else:
                        thecommand += "(is %s %s %s)\n" %(block1, block2, relation)            
        
        #Multiple blocks and multiple relations
        #Assumptions is that each pair of blocks goes with one relation: 2 to 1
        if len(blocks)>3 and len(relations)>1:
            #Grap 2 blocks at a time
            rel_count = 0
            for i, block in enumerate(blocks):
                mod = i % 2
                if mod == 0:
                    block1=block
                else:
                    block2=block
                    if relations[rel_count] == "bottom-of":# swap_on_top_of:
                        thecommand += "(is %s %s %s)\n" %(block2, block1, "on-top-of")
                    else:
                        thecommand += "(is %s %s %s)\n" %(block1, block2, relations[rel_count])

                    if rel_count < len(relations):
                        rel_count+=1

        #print("Block1: " + block1 + " Block2: " + block2)

        #Create the command for an is relation
        #if block1 and block2 and relation:
        #    if relations[rel_count] == "bottom-of":#swap_on_top_of:
        #        thecommand = "(is %s %s %s)\n" %(block2, block1, "on-top-of")
        #    else:
        #        thecommand = "(is %s %s %s)\n" %(block1, block2, relation)

        if block1 and color:
            thecommand = "(has %s color %s)\n" % (block1,color)

        if block1 and location and (len(location)==3):
            thecommand = "(has %s location %s %s %s)\n" % (block1,location[0],location[1],location[2])

        #pdb.set_trace()
        if thecommand != "":
            print("Storing Command(s): " + thecommand)
            with open(outputfile, "a") as myfile:
                myfile.write(thecommand)
            return "SUCCESS"
    except:
        return "FAILURE"

In [15]:
def partsOfSpeechTag(words, outputfile):
    
    #Part of Speech tagging    
    parts_speech = nltk.pos_tag(words)
    #print(parts_speech)
    #parts_of_speech.append(parts_speach)
        
    #print("\nParts of Speech for each sentence")
    #for p in parts_speech:
    #print("\nThe Parts of Speech:")
    print("Tokenized Words: " + str(words))
    enumerate_parts(parts_speech)
    print("Parts of Speech: " + str(parts_speech))
    tree = chunkParts(parts_speech)
    
    location = []
    block_1 = ""
    block_2 = ""
    color = ""
    relation = ""
    
    hasAnd = processAndInSentence(parts_speech)
    
    #Check for location value
    location = processLocation(parts_speech)
    
    #Array of blocks, allow for something like: Block 1 is next to block2 and block3.
    #Need to generate more than one command for that scenario
    #Will always assume the first block is the key block: Block 1 is next to block 2 and block3 is next to block4 will not work.
    #Maybe add something to catch the "and", if there are 4 blocks it can generate 2 commands.
    blocks = checkBlocks(parts_speech)
    
    #Set the type of relation
    relation = setRelation(parts_speech)
    
    #Set the color
    color = processColor(parts_speech)
    
    return format_command(outputfile, blocks, relation, color, location, hasAnd) 


In [16]:
def readinfile(inputfile, outputfile):
    #os.remove(inputfile) if os.path.exists(inputfile) else None
    os.remove(outputfile) if os.path.exists(outputfile) else None

    commandFile = open(inputfile) 
    data = commandFile.read()# Use this to read file content as a stream: 
    commandFile.close()
    
    cfdist = ConditionalFreqDist()
    
    sentences = sent_tokenize(data)
    #print(sentences)
    #parts_of_speech = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        temp = []
        for w in words:
            temp.append(w.lower())
        words = temp[0:-1]
        
        #Parts of Speech tagging
        if(partsOfSpeechTag(words, outputfile)=="SUCCESS"):
        
        #if(format_command(outputfile, block_1, block_2, relation, color, location, swap_on_top_of)=="SUCCESS"):
             condition = len(words)
             cfdist[condition]["SUCCESS"] += 1
             #print(condition)
        else:
             condition = len(words)
             cfdist[condition]["Fail"] += 1
             #print(condition)
        print()
    
    for f in cfdist:
        for k in cfdist[f]:
            print(str(f)+": " +str(k))
            #print("Frequency of", f, cfdist.freq(f))

Code for Speech Reecognition

In [17]:
r = sr.Recognizer()
recording = sr.AudioFile('Recording.wav')
with recording as source:
    audio = r.record(source)
recording_str = r.recognize_google(audio)
new_file=open("nlp_tests/Recording.txt",mode="w",encoding="utf-8")
new_file.write(recording_str)
new_file.close()

In [18]:
#Read in initial file
outputStart = "aStarSearch/nlp_outputs/start1.txt"
readinfile("nlp_tests/start1.txt", outputStart)

#Read in goal file
outputGoal = "aStarSearch/nlp_outputs/goal1.txt"
readinfile("nlp_tests/goal1.txt", outputGoal)


Tokenized Words: ['block', '1', 'has', 'color', 'red']
Parts of Speech: [('block', 'BNN'), ('1', 'CD'), ('has', 'VBZ'), ('color', 'NN'), ('red', 'VARC')]
Storing Command(s): (has block1 color red)


Tokenized Words: ['block2', 'has', 'location', '8', '3', '1']
Parts of Speech: [('block2', 'BNN'), ('has', 'VBZ'), ('location', 'VARL'), ('8', 'CD'), ('3', 'CD'), ('1', 'CD')]
Storing Command(s): (has block2 location 8 3 1)


Tokenized Words: ['block3', 'has', 'color', 'green']
Parts of Speech: [('block3', 'BNN'), ('has', 'VBZ'), ('color', 'NN'), ('green', 'VARC')]
Storing Command(s): (has block3 color green)


Tokenized Words: ['block1', 'is', 'at', 'the', 'spot', '8', '3', '0']
Parts of Speech: [('block1', 'BNN'), ('is', 'VARL'), ('at', 'VARL'), ('the', 'DT'), ('spot', 'VARL'), ('8', 'CD'), ('3', 'CD'), ('0', 'CD')]
Storing Command(s): (has block1 location 8 3 0)


Tokenized Words: ['block3', 'location', 'is', '8', '3', '2']
Parts of Speech: [('block3', 'BNN'), ('location', 'VARL'), ('is'

In [ ]:
#!{sys.executable} -m pip install pandas
#import pandas as pd
%run -i "./aStarSearch/aStarSearch.py" $outputStart $outputGoal

aStarSearch/nlp_outputs/start1.txt
aStarSearch/nlp_outputs/goal1.txt

Searching...

END STATE:
        Coordinates Properties
Block                         
block1    (4, 1, 0)      [red]
block2    (4, 0, 0)         []
block21   (3, 5, 0)         []
block23   (3, 4, 0)         []
block3    (4, 2, 0)    [green]
['command slide block21 -1 -1', 'command slide block21 -1 1', 'command slide block21 -1 1', 'command slide block21 -1 -1', 'command slide block21 -1 0', 'command slide block1 -1 -1', 'command slide block1 -1 -1', 'command slide block1 0 -1', 'command slide block23 -1 1', 'command slide block23 -1 1', 'command slide block23 -1 -1', 'command slide block23 -1 0', 'command slide block1 -1 0', 'command slide block23 -1 -1', 'command grab block3', 'command carry block3 0 1 -1', 'command carry block3 0 0 -1', 'command release block3', 'command grab block2', 'command carry block2 -1 0 -1', 'command release block2', 'command grab block2', 'command release block2', 'command slide block1 -1